## Hello World SC with Membership number
#### 07.2 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-09

* Smart contract interaction with blockchain
* Smart contract depends on user's coin holdings

## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding the code below

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19907242


In [7]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


In [13]:
asset_holdings_df(algod_client, Dina['public'])

,amount,unit,asset-id,name,decimals
0,1.377,ALGO,0,Algorand,6
1,997.000,VtC,70161280,VoteCoin,2
2,997.000,VtC,70166124,VoteCoin,2


### Some helper functions

In [6]:
def read_local_state(client, addr, app_id):
    # reads a user's local state
    # client = algod_client
    # addr = public addr of the user that we want to inspect
    results = client.account_info(addr)
    for local_state in results["apps-local-state"]:
        if local_state["id"] == app_id:
            if "key-value" not in local_state:
                return {}
            return format_state(local_state["key-value"])
    return {}

def read_global_state(client, app_id):
    # reads an app's global state
    return  algod_client.application_info(app_id)["params"]["global-state"]

def format_state(state):
    # formats the state (local/global) nicely 
    formatted = {}
    textvariables = {'Info','Note'}        # <---- update this! (List of Text variables in SC)
    for item in state:
        key = base64.b64decode(item["key"]).decode("utf-8")
        value = item["value"]
        if value["type"] == 1:
            if key in textvariables:                 # Format text variables
                formatted_value = base64.b64decode(value["bytes"]).decode("utf-8")
            else:                                    # Format addresses
                formatted_value = base64.b32encode(base64.b64decode(value["bytes"]))
            formatted[key] = formatted_value
        else:
            formatted[key] = value["uint"]
    return formatted

## A first stateful smart contract
The stateful smart contract consists of two parts
* The `approval_program` that handles everything except opt out
* The `clear_state_program` that handles the opt out

## Our first project: HelloWorld with Membership number
* Idea: just write "Hello World" into the global state
* We also count the number of members
* Every new member gets a membership number
* Do not do anything else

#### Step 1: Define Approval program
This is the program that handles all interactions except opt out:
* Creation of the SC `handle_creation`
* Opt-in of individual users `handle_register`
* Interactions (calls) of individual users `handle_interact`


In [ ]:
handle_creation = Seq(
    [
        # Initialize a GLOBAL variable
        App.globalPut(Bytes("Note"),       Bytes("Hello world!")),
        App.globalPut(Bytes("Members"),    Int(0)),
        # Return "OK"
        Return(Int(1))        
    ]
)

handle_optin = Seq(
    [
        App.localPut(
            Int(0),                                     # which user? current one!
            Bytes("MembershipNo"),                      # key
            App.globalGet(Bytes("Members")) + Int(1)    # value: Membership number = current members + 1
        ),
        # Increase number of Members
        App.globalPut( Bytes("Members"),                             # key
                       App.globalGet(Bytes("Members")) + Int(1)      # value
                     ),
        # Return "OK"
        App.localPut(Int(0), Bytes("YourHolding"), Balance(Txn.sender())),
        If(Balance(Txn.sender())<Int(5000000),
            App.localPut(Int(0), Bytes("You_are"),  Bytes("poor")),
            App.localPut(Int(0), Bytes("You_are"),  Bytes("rich"))
          ),
        App.localPut(Int(0), Bytes("YourHolding"), Balance(Txn.sender())),

        Return(Int(1))        
    ]
)


handle_closeout = Return( Int(1) )                # Not doing anything, returning "OK"  

handle_updateapp = Return( Int(0) )               # Always FALSE ... updating not allowed

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

# handle interaction
handle_noop  = Return ( Int(1) )                   # Not doing anything, returning "OK"           

In [ ]:
hello_approval_pyteal = Cond(
    [Txn.application_id() == Int(0),              handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,    handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut, handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp, handle_noop],
)

#### Compile PyTEAL -> TEAL
* Notice the `Mode.Application` (was `Mode.Signature`)

In [ ]:
hello_approval_teal = compileTeal(hello_approval_pyteal,mode=Mode.Application, version=3)
print(hello_approval_teal)

#### Step 1b: Define Clear State program
* This program handles forced opt-outs

In [ ]:
hello_clear_pyteal =  Return(Int(1))    # not doing anything

In [ ]:
hello_clear_teal = compileTeal(hello_clear_pyteal,mode=Mode.Application, version=3)
print(hello_clear_teal)

#### Compile TEAL -> Bytecode
This is slightly different ... we need one additional step for Byte-encoding

In [ ]:
hello_approval_b64 = algod_client.compile(hello_approval_teal)
Hello_Approval =  base64.b64decode(hello_approval_b64['result'])

hello_clear_b64 = algod_client.compile(hello_clear_teal)
Hello_Clear =  base64.b64decode(hello_clear_b64['result'])

## Deploy Smart Contract

##### Bob deploys the smart contract
* Reserve global storage with `StateSchema`
* New command `ApplicationCreateTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCreateTxn)

In [ ]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# How much space do we need?
global_ints = 1    # One global numeric variable
global_bytes = 1   # for Note
hello_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 1     # One local numeric variable
local_bytes = 0    # No local text variable
hello_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = Bob['public'],              # <-- sender public
      sp = sp,                             # <-- sp
      on_complete = 0,                     # <-- when finished do nothing
      approval_program = Hello_Approval,   # <-- approval program 
      clear_program = Hello_Clear,         # <-- clear program 
      global_schema = hello_global_schema, # <-- reserve global space 
      local_schema = hello_local_schema    # <-- reserve local space
    )

In [ ]:
# Step 2: sign transaction
stxn = txn.sign(Bob['private'])

# Step 3: send
txid=algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

## The Smart Contract is now deployed
* And there is alreasdy something to see

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
# Program code immediately visible on the web
print('https://testnet.algoexplorer.io/application/{}'.format(app_id))

## Using the Smart Contract (1): Users opt-in
* Nothing will happen, but we can test everything
* Using `ApplicationOptInTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationOptInTxn)

In [ ]:
user = Charlie

# Step 1: prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)

# Step 2: sign transaction
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Watch the state of the contract evolve

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,user['public'],app_id)

## Users can call the Smart contract ("make a visit")
* Does not really make sense here ...

## Inspect the global state and the local states of the users

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,Alice['public'],app_id)

In [ ]:
read_local_state(algod_client,Bob['public'],app_id)

In [ ]:
read_local_state(algod_client,Charlie['public'],app_id)

In [ ]:
read_local_state(algod_client,Dina['public'],app_id)

## Users close out (leave) App
* With a `ApplicationCloseOutTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCloseOutTxn)

In [ ]:
user = Alice

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationCloseOutTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

## Inspect the global state and the local states of the users

In [ ]:
# Note that we do not update the number of members
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,Alice['public'],app_id)

In [ ]:
read_local_state(algod_client,Bob['public'],app_id)

In [ ]:
read_local_state(algod_client,Charlie['public'],app_id)

In [ ]:
read_local_state(algod_client,Dina['public'],app_id)

## Deleting the app
* Rather important, because an address can only create **10 apps**
* App can be deleted by creator

In [ ]:
creator = Bob

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])